In [1]:

import torch
import os

import numpy as np

from monai.losses import DiceCELoss

from monai.data import Dataset, DataLoader
from monai.transforms import LoadImage, Resized, Compose, Lambda
from monai.networks.nets import UNETR
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
dataset_list = []

# read all folders in Images

base_path = "./Task01_BrainTumour"


for file in os.listdir(os.path.join(base_path, "imagesTr")):

    data_dict = {
        "img": os.path.join(base_path, "imagesTr", file), 
        "seg": os.path.join(base_path, "labelsTr", file)
    }

    dataset_list.append(data_dict)

In [4]:
class BrainTumor(Dataset):

    def __init__(self, data, transform=None):
        super().__init__(data, transform)
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        element_dict = self.data[index]
        img_name = element_dict["img"]
        seg_name = element_dict["seg"]
        img = LoadImage(image_only=True, ensure_channel_first=True, simple_keys=True)(img_name)
        seg = LoadImage(image_only=True, ensure_channel_first=True, simple_keys=True)(seg_name)
        if self.transform:
            transformed = self.transform({"img": img, "seg": seg} )
            img = transformed["img"]
            seg = transformed["seg"]
            
        seg = seg.type(torch.LongTensor)
            
        return img, seg

In [5]:
train_transforms = Compose([
    Lambda(lambda x: {"img": x["img"].permute(0, 3, 1, 2), "seg": x["seg"].permute(0, 3, 1, 2)}),
    Resized(keys=["img", "seg"], spatial_size=(32, 64, 64), mode=('nearest')),
    #ToTensorD(keys=["img", "seg"])
])

In [6]:
dataset = BrainTumor(dataset_list, transform=train_transforms)
print(len(dataset))

train_dataset, test_dataset = torch.utils.data.random_split(dataset, [390, 94])

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

484


In [14]:
model = UNETR(
    in_channels=4,
    out_channels=4,
    img_size=(32, 64, 64),
    feature_size=16,
    hidden_size=768,
    mlp_dim=3072,
    num_heads=6,
    norm_name="instance",
    res_block=True,
    dropout_rate=0.0,
    conv_block=False,
).to(device)
print(next(model.parameters()).device)

print(model.state_dict().keys())

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacty of 15.69 GiB of which 40.00 MiB is free. Process 11737 has 3.39 GiB memory in use. Process 14641 has 7.93 GiB memory in use. Including non-PyTorch memory, this process has 3.96 GiB memory in use. Of the allocated memory 3.57 GiB is allocated by PyTorch, and 107.25 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [9]:
def calculate_iou(predicted, target):
    """
    Calculate Intersection over Union (IoU) for binary segmentation.

    Args:
    - predicted (torch.Tensor): Predicted binary mask (0 or 1).
    - target (torch.Tensor): Ground truth binary mask (0 or 1).

    Returns:
    - float: IoU score.
    """
    intersection = torch.logical_and(predicted, target).sum().item()
    union = torch.logical_or(predicted, target).sum().item()

    iou = intersection / union if union != 0 else 0.0
    return iou

In [10]:
loss_function = DiceCELoss(to_onehot_y=True, softmax=True) #torch.nn.MSELoss()
optimizer = torch.optim.AdamW(model.parameters(), 0.1)

In [12]:
num_epochs = 3
losses = []
test_losses = []
ious = []
test_ious = []

for epoch in range(num_epochs):
    
    epoch_loss = 0
    epoch_test_loss = 0
    epoch_iou = 0
    epoch_test_iou = 0
    
    model.train() 
    
    for i, (img, seg) in enumerate(tqdm(train_dataloader)):
        
        img = img.to(device)
        seg = seg.to(device)
        
        optimizer.zero_grad()
        
        output = model(img)
        
        iou = calculate_iou(output, seg)
        ious.append(iou)
        
        loss = loss_function(output, seg)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    losses.append(epoch_loss)
    
    model.eval()
    with torch.no_grad():
        for i, (img, seg) in enumerate(test_dataloader):
            
            img = img.to(device)
            seg = seg.to(device)
             
            output = model(img)
            
            iou = calculate_iou(output, seg)
            test_ious.append(iou)
            
            loss = loss_function(output, seg)
            
            epoch_test_loss += loss.item()
            
        test_losses.append(epoch_test_loss)
        
        print(f"""
              Epoch: {epoch} 
              | Train Loss: {epoch_loss / len(train_dataloader)} 
              | Test Loss: {epoch_test_loss / len(test_dataloader)} 
              | Train IoU: {ious[-1]} | Test IoU: {test_ious[-1]}
            """)

  0%|          | 0/13 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f436a18fc70>
Traceback (most recent call last):
  File "/home/erikmyhre/miniconda3/envs/monai/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1478, in __del__
    self._shutdown_workers()
  File "/home/erikmyhre/miniconda3/envs/monai/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1442, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/home/erikmyhre/miniconda3/envs/monai/lib/python3.10/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
  File "/home/erikmyhre/miniconda3/envs/monai/lib/python3.10/multiprocessing/popen_fork.py", line 40, in wait
    if not wait([self.sentinel], timeout):
  File "/home/erikmyhre/miniconda3/envs/monai/lib/python3.10/multiprocessing/connection.py", line 931, in wait
    ready = selector.select(timeout)
  File "/home/erikmyhre/miniconda3/envs/monai/lib/python3.10/selectors.py", lin

OutOfMemoryError: CUDA out of memory. Tried to allocate 512.00 MiB. GPU 0 has a total capacty of 15.69 GiB of which 346.94 MiB is free. Process 11737 has 3.39 GiB memory in use. Process 14641 has 7.93 GiB memory in use. Including non-PyTorch memory, this process has 3.69 GiB memory in use. Of the allocated memory 3.23 GiB is allocated by PyTorch, and 174.55 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF